<a href="https://colab.research.google.com/github/JadeEmm/hugging-face-agents-course/blob/main/units/en/unit2/smolagents/code_agents_160032025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents That Use Code
This notebook is to develop a multi-step `CodeAgent` that operates.


## Install the dependencies and login to HF account to access the Inference API

In [1]:
#Install `smolagents`
!pip install smolagents -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [2]:
#Login to Hugging Face hub to access the inference API
from huggingface_hub import notebook_login
notebook_login()

## Building an AI-Powered Learning Tutor with Hugging Face and SmolAgents

Build an AI-powered learning tutor using Hugging Face's inference API and SmolAgents.
- Creating tools to develop an agent that can:

- Explain concepts to children based on their age

- Generate quizzes dynamically

- Check spelling and grammar

- Provide fun facts

The model used is `HfApiModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Here, we use the `@tool` decorator to define a custom function that acts as a tool.

Building an Educational Assistant with smolagents
This notebook demonstrates how to create an AI-powered educational assistant using smolagents. The assistant is designed to help children learn by explaining concepts, generating quizzes, checking spelling, providing fun facts, and managing study time.


What is smolagents?
smolagents is a library that specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.
Using Python Imports Inside the Agent
Code execution has strict security measures - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in additional_authorized_imports.
For our educational assistant, we'll authorize several useful libraries:

datetime - For time tracking and scheduling
math - For calculations in quizzes and scoring
random - For generating varied learning content
json - For structured data formatting

In [29]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool
import datetime
import math
import random
import json

# Use Hugging Face's inference API model
# model = HfApiModel(model="Qwen/Qwen2.5-Coder-32B-Instruct") # moved this inside get_agent

### TOOL 1: Explain Concepts at Age-Appropriate Levels (Using AI)
@tool
def explain_concept_for_age(topic: str, age: int) -> str:
    """
    Uses an AI model from Hugging Face to generate an age-appropriate explanation of a given topic.
    Args:
        topic: The subject the child is learning (e.g., 'math', 'science', 'history').
        age: The child's age (should be between 5 and 11).
    """
    # Inject model instance from agent
    model = agent.model
    prompt = f"Explain {topic} in a way that a {age}-year-old child can understand. Use simple language and examples."
    return model.invoke(prompt)

### TOOL 2: Generate an Interactive Quiz (Using AI)
@tool
def generate_quiz(subject: str, age: int) -> dict:
    """
    Uses an AI model to create a quiz with multiple-choice questions for a given subject and age.
    Args:
        subject: The subject for the quiz (e.g., 'math', 'science', 'english').
        age: The child's age (5-11).
    """
    # Inject model instance from agent
    model = agent.model
    prompt = f"""Create a 3-question multiple-choice quiz on {subject} for a {age}-year-old.
    Format as a JSON array with this structure for each question:
    {{"question": "Question text", "options": ["A. Option1", "B. Option2", "C. Option3", "D. Option4"], "correct": "A"}}
    """
    quiz_response = model.invoke(prompt)
    try:
        # Try to parse the response as JSON
        quiz_data = json.loads(quiz_response)
        return {"quiz": quiz_data}
    except:
        # Fallback if JSON parsing fails
        return {"quiz": quiz_response}

### TOOL 3: Spelling & Grammar Correction
@tool
def check_spelling(sentence: str) -> str:
    """
    Uses AI to correct spelling and grammar in a given sentence.
    Args:
        sentence: A short sentence provided by the user.
    """
    # Inject model instance from agent
    model = agent.model
    prompt = f"Correct the grammar and spelling in this sentence: '{sentence}'"
    return model.invoke(prompt)

### TOOL 4: Fun Facts Generator
@tool
def get_fun_fact(subject: str) -> str:
    """
    Uses AI to generate a fun and accurate educational fact related to a given subject or topic.
    Args:
        subject: The subject (e.g., 'math', 'science', 'history').
    """
    # Inject model instance from agent
    model = agent.model
    prompt = f"Give me an interesting and fun fact about {subject} that a child would enjoy."
    return model.invoke(prompt)


### TOOL 5: Study Time Tracker - This tool uses the datetime module to help children track their study sessions
@tool
def track_study_time(subject: str, duration_minutes: int) -> str:
    """
    Tracks study time for a subject and provides encouragement.
    Args:
        subject: The subject being studied.
        duration_minutes: How many minutes spent studying.
    """
    now = datetime.datetime.now()
    end_time = now + datetime.timedelta(minutes=duration_minutes)

    # Calculate a "focus score" based on time spent (just for fun)
    focus_score = min(100, duration_minutes * 2)

    return {
        "start_time": now.strftime("%H:%M"),
        "end_time": end_time.strftime("%H:%M"),
        "subject": subject,
        "duration": duration_minutes,
        "focus_score": focus_score,
        "message": f"Great job studying {subject} for {duration_minutes} minutes! Your focus score is {focus_score}/100."
    }

### TOOL 6: Quiz Scorer - This tool uses math to calculate quiz scores and percentages
@tool
def score_quiz(answers: str, correct_answers: str) -> dict:
    """
    Scores a quiz based on user answers and correct answers.
    Args:
        answers: Comma-separated answers (e.g., "A,B,C").
        correct_answers: Comma-separated correct answers (e.g., "A,C,C").
    """
    user_answers = [a.strip().upper() for a in answers.split(",")]
    correct = [a.strip().upper() for a in correct_answers.split(",")]

    if len(user_answers) != len(correct):
        return {"error": "Number of answers doesn't match number of questions"}

    # Calculate results
    num_correct = sum(1 for ua, ca in zip(user_answers, correct) if ua == ca)
    total = len(correct)
    percentage = (num_correct / total) * 100

    # Generate feedback based on performance
    if percentage >= 90:
        feedback = "Excellent! You're a superstar!"
    elif percentage >= 75:
        feedback = "Great job! Keep up the good work!"
    elif percentage >= 50:
        feedback = "Good effort! Let's review the ones you missed."
    else:
        feedback = "Let's review these together. You'll get it next time!"

    return {
        "score": f"{num_correct}/{total}",
        "percentage": f"{percentage:.1f}%",
        "feedback": feedback,
        "correct_questions": [i+1 for i, (ua, ca) in enumerate(zip(user_answers, correct)) if ua == ca],
        "incorrect_questions": [i+1 for i, (ua, ca) in enumerate(zip(user_answers, correct)) if ua != ca]
    }

### TOOL 7: Learning Schedule Generator - This tool uses datetime to create optimized study schedules
@tool
def generate_learning_schedule(subjects: str, available_hours: float) -> dict:
    """
    Creates an optimized learning schedule based on subjects and available time.
    Args:
        subjects: Comma-separated list of subjects to study.
        available_hours: Total hours available for studying.
    """
    subject_list = [s.strip() for s in subjects.split(",")]

    # Calculate time allocation
    available_minutes = int(available_hours * 60)
    subject_count = len(subject_list)

    # Base minutes per subject
    base_minutes = available_minutes // subject_count

    # Distribute remaining minutes
    remaining = available_minutes - (base_minutes * subject_count)

    schedule = []
    start_time = datetime.datetime.now()

    for i, subject in enumerate(subject_list):
        # Allocate slightly more time to first subjects if there are remaining minutes
        subject_minutes = base_minutes + (1 if i < remaining else 0)

        # Add short breaks after each subject (except the last one)
        break_minutes = 5 if i < subject_count - 1 else 0

        end_time = start_time + datetime.timedelta(minutes=subject_minutes)

        schedule.append({
            "subject": subject,
            "start_time": start_time.strftime("%H:%M"),
            "end_time": end_time.strftime("%H:%M"),
            "duration": f"{subject_minutes} minutes"
        })

        # Update start time for next subject (including break)
        start_time = end_time + datetime.timedelta(minutes=break_minutes)

    return {
        "schedule": schedule,
        "total_study_time": f"{available_hours:.1f} hours",
        "includes_breaks": "Yes, 5-minute breaks between subjects"
    }

# Create the agent with all tools and authorized imports
agent = CodeAgent(
    tools=[
        explain_concept_for_age,
        generate_quiz,
        check_spelling,
        get_fun_fact,
        track_study_time,
        score_quiz,
        generate_learning_schedule,
        DuckDuckGoSearchTool()
    ],
    model=HfApiModel(model="Qwen/Qwen2.5-Coder-32B-Instruct"), # define model here.
    additional_authorized_imports=['datetime', 'math', 'random', 'json']
)

# Example usage
if __name__ == "__main__":
    print(agent.run("Explain dinosaurs to a 6-year-old."))
    print(agent.run("Generate a science quiz for a 9-year-old."))
    print(agent.run("Score my quiz. My answers were A,B,C and the correct answers are A,C,C."))
    print(agent.run("Track my study time for math for 45 minutes."))
    print(agent.run("Create a learning schedule for math, science, reading with 2 hours available."))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Explain dinosaurs to a 6-year-old.                                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  explanation = explain_concept_for_age(topic="dinosaurs", age=6)                                                  
  print(explanation)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'explanation = explain_concept_for_age(topic="dinosaurs", age=6)' due to: 
AttributeError: 'HfApiModel' object has no attribute 'invoke'

[Step 1: Duration 0.31 seconds| Input tokens: 2,620 | Output tokens: 61]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import json                                                                                                      
  schedule = generate_learning_schedule(subjects="math,science,reading", available_hours=2)                        
  final_answer(json.dumps(schedule, indent=2))                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {
  "schedule": [
    {
      "subject": "math",
      "start_time": "14:02",
      "end_time": "14:42",
      "duration": "40 minutes"
    },
    {
      "subject": "science",
      "start_time": "14:47",
      "end_time": "15:27",
      "duration": "40 minutes"
    },
    {
      "subject": "reading",
      "start_time": "15:32",
      "end_time": "16:12",
      "duration": "40 minutes"
    }
  ],
  "total_study_time": "2.0 hours",
  "includes_breaks": "Yes, 5-minute breaks between subjects"
}

[Step 4: Duration 3.78 seconds| Input tokens: 12,146 | Output tokens: 280]

{
  "schedule": [
    {
      "subject": "math",
      "start_time": "14:02",
      "end_time": "14:42",
      "duration": "40 minutes"
    },
    {
      "subject": "science",
      "start_time": "14:47",
      "end_time": "15:27",
      "duration": "40 minutes"
    },
    {
      "subject": "reading",
      "start_time": "15:32",
      "end_time": "16:12",
      "duration": "40 minutes"
    }
  ],
  "total_study_time": "2.0 hours",
  "includes_breaks": "Yes, 5-minute breaks between subjects"
}


## Sharing Agent to the Hub


The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:


app.py file

In [12]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [30]:
import streamlit as st
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool
import datetime
import math
import random
import json

# Set page title and configuration
st.set_page_config(page_title="Learning Buddy - Educational Assistant", layout="wide")
st.title("Learning Buddy")
st.subheader("Your AI Educational Assistant for Kids")

# Initialize the model (you'll need to provide an API key)
@st.cache_resource
def get_model():
    return HfApiModel(model="Qwen/Qwen2.5-Coder-32B-Instruct")

model = get_model()

# Import your tools from the original code
@tool
def explain_concept_for_age(topic: str, age: int) -> str:
    """
    Uses an AI model from Hugging Face to generate an age-appropriate explanation of a given topic.
    Args:
        topic: The subject the child is learning (e.g., 'math', 'science', 'history').
        age: The child's age (should be between 5 and 11).
    """
    prompt = f"Explain {topic} in a way that a {age}-year-old child can understand. Use simple language and examples."
    return model.invoke(prompt)

### TOOL 2: Generate an Interactive Quiz (Using AI)
@tool
def generate_quiz(subject: str, age: int) -> dict:
    """
    Uses an AI model to create a quiz with multiple-choice questions for a given subject and age.
    Args:
        subject: The subject for the quiz (e.g., 'math', 'science', 'english').
        age: The child's age (5-11).
    """
    prompt = f"""Create a 3-question multiple-choice quiz on {subject} for a {age}-year-old.
    Format as a JSON array with this structure for each question:
    {{"question": "Question text", "options": ["A. Option1", "B. Option2", "C. Option3", "D. Option4"], "correct": "A"}}
    """
    quiz_response = model.invoke(prompt)
    try:
        # Try to parse the response as JSON
        quiz_data = json.loads(quiz_response)
        return {"quiz": quiz_data}
    except:
        # Fallback if JSON parsing fails
        return {"quiz": quiz_response}

### TOOL 3: Spelling & Grammar Correction
@tool
def check_spelling(sentence: str) -> str:
    """
    Uses AI to correct spelling and grammar in a given sentence.
    Args:
        sentence: A short sentence provided by the user.
    """
    prompt = f"Correct the grammar and spelling in this sentence: '{sentence}'"
    return model.invoke(prompt)

### TOOL 4: Fun Facts Generator
@tool
def get_fun_fact(subject: str) -> str:
    """
    Uses AI to generate a fun and accurate educational fact related to a given subject or topic.
    Args:
        subject: The subject (e.g., 'math', 'science', 'history').
    """
    prompt = f"Give me an interesting and fun fact about {subject} that a child would enjoy."
    return model.invoke(prompt)

### TOOL 5: Study Time Tracker - This tool uses the datetime module to help children track their study sessions
@tool
def track_study_time(subject: str, duration_minutes: int) -> str:
    """
    Tracks study time for a subject and provides encouragement.
    Args:
        subject: The subject being studied.
        duration_minutes: How many minutes spent studying.
    """
    now = datetime.datetime.now()
    end_time = now + datetime.timedelta(minutes=duration_minutes)

    # Calculate a "focus score" based on time spent (just for fun)
    focus_score = min(100, duration_minutes * 2)

    return {
        "start_time": now.strftime("%H:%M"),
        "end_time": end_time.strftime("%H:%M"),
        "subject": subject,
        "duration": duration_minutes,
        "focus_score": focus_score,
        "message": f"Great job studying {subject} for {duration_minutes} minutes! Your focus score is {focus_score}/100."
    }

### TOOL 6: Quiz Scorer - This tool uses math to calculate quiz scores and percentages
@tool
def score_quiz(answers: str, correct_answers: str) -> dict:
    """
    Scores a quiz based on user answers and correct answers.
    Args:
        answers: Comma-separated answers (e.g., "A,B,C").
        correct_answers: Comma-separated correct answers (e.g., "A,C,C").
    """
    user_answers = [a.strip().upper() for a in answers.split(",")]
    correct = [a.strip().upper() for a in correct_answers.split(",")]

    if len(user_answers) != len(correct):
        return {"error": "Number of answers doesn't match number of questions"}

    # Calculate results
    num_correct = sum(1 for ua, ca in zip(user_answers, correct) if ua == ca)
    total = len(correct)
    percentage = (num_correct / total) * 100

    # Generate feedback based on performance
    if percentage >= 90:
        feedback = "Excellent! You're a superstar!"
    elif percentage >= 75:
        feedback = "Great job! Keep up the good work!"
    elif percentage >= 50:
        feedback = "Good effort! Let's review the ones you missed."
    else:
        feedback = "Let's review these together. You'll get it next time!"

    return {
        "score": f"{num_correct}/{total}",
        "percentage": f"{percentage:.1f}%",
        "feedback": feedback,
        "correct_questions": [i+1 for i, (ua, ca) in enumerate(zip(user_answers, correct)) if ua == ca],
        "incorrect_questions": [i+1 for i, (ua, ca) in enumerate(zip(user_answers, correct)) if ua != ca]
    }

### TOOL 7: Learning Schedule Generator - This tool uses datetime to create optimized study schedules
@tool
def generate_learning_schedule(subjects: str, available_hours: float) -> dict:
    """
    Creates an optimized learning schedule based on subjects and available time.
    Args:
        subjects: Comma-separated list of subjects to study.
        available_hours: Total hours available for studying.
    """
    subject_list = [s.strip() for s in subjects.split(",")]

    # Calculate time allocation
    available_minutes = int(available_hours * 60)
    subject_count = len(subject_list)

    # Base minutes per subject
    base_minutes = available_minutes // subject_count

    # Distribute remaining minutes
    remaining = available_minutes - (base_minutes * subject_count)

    schedule = []
    start_time = datetime.datetime.now()

    for i, subject in enumerate(subject_list):
        # Allocate slightly more time to first subjects if there are remaining minutes
        subject_minutes = base_minutes + (1 if i < remaining else 0)

        # Add short breaks after each subject (except the last one)
        break_minutes = 5 if i < subject_count - 1 else 0

        end_time = start_time + datetime.timedelta(minutes=subject_minutes)

        schedule.append({
            "subject": subject,
            "start_time": start_time.strftime("%H:%M"),
            "end_time": end_time.strftime("%H:%M"),
            "duration": f"{subject_minutes} minutes"
        })

        # Update start time for next subject (including break)
        start_time = end_time + datetime.timedelta(minutes=break_minutes)

    return {
        "schedule": schedule,
        "total_study_time": f"{available_hours:.1f} hours",
        "includes_breaks": "Yes, 5-minute breaks between subjects"
    }

# Create the agent
@st.cache_resource
def get_agent():
    return CodeAgent(
        tools=[
            explain_concept_for_age,
            generate_quiz,
            check_spelling,
            get_fun_fact,
            track_study_time,
            score_quiz,
            generate_learning_schedule,
            DuckDuckGoSearchTool()
        ],
        model=HfApiModel(),
        additional_authorized_imports=['datetime', 'math', 'random', 'json'],
        max_steps=10,
        verbosity_level=2
    )

# UI Elements
st.markdown("### Ask your Learning Buddy")

# Input area
user_input = st.text_area("Type your question here:",
                         placeholder="Example: Explain dinosaurs to a 6-year-old")

# Age selector
age = st.slider("Child's age:", min_value=5, max_value=11, value=8)

# Add a button to process the request
if st.button("Get Answer") and user_input:
    with st.spinner("Learning Buddy is thinking..."):
        # Add age context to the query if not explicitly mentioned
        if "age" not in user_input.lower() and "year old" not in user_input.lower():
            query = f"{user_input} for a {age}-year-old."
        else:
            query = user_input

        # Get response from agent
        agent = get_agent()
        response = agent.run(query)

        # Display the response
        st.markdown("### Answer:")
        st.markdown(response)

# Display some example questions
with st.expander("Example questions you can ask"):
    st.markdown("""
    - Explain dinosaurs to a 6-year-old
    - Generate a science quiz for a 9-year-old
    - Check spelling in "I went too the park yesterdy"
    - Tell me a fun fact about space
    - Plan my study time for math for 30 minutes
    - Score my quiz. My answers were A,B,C and the correct answers are A,C,C
    - Create a learning schedule for math, science, reading with 2 hours available
    """)

2025-03-16 14:02:55.642 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.643 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.645 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-16 14:02:55.662 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [31]:
agent.run("Explain why plants need sunlight for 7 year old")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Explain why plants need sunlight for 7 year old                                                                 │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  explanation = explain_concept_for_age(topic="why plants need sunlight", age=7)                                   
  final_answer(explanation)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'explanation = explain_concept_for_age(topic="why plants need sunlight", age=7)' due 
to: AttributeError: 'HfApiModel' object has no attribute 'invoke'

[Step 1: Duration 0.13 seconds| Input tokens: 2,621 | Output tokens: 67]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  explanation = "Plants need sunlight to make their food through a process called photosynthesis. Sunlight helps   
  plants turn air and water into energy, which makes them grow big and strong, just like how you need food to      
  grow!"                                                                                                           
  final_answer(explanation)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Plants need sunlight to make their food through a process called photosynthesis. Sunlight helps
plants turn air and water into energy, which makes them grow big and strong, just like how you need food to grow!

[Step 2: Duration 0.12 seconds| Input tokens: 5,458 | Output tokens: 178]

'Plants need sunlight to make their food through a process called photosynthesis. Sunlight helps plants turn air and water into energy, which makes them grow big and strong, just like how you need food to grow!'

# Langfuse

In [ ]:
import os
import base64
from google.colab import userdata

LANGFUSE_PUBLIC_KEY=userdata.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=userdata.get("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(tools=[], model=HfApiModel())
learning_buddy_agent = agent.from_hub('Jade-E/Party_Planner_Agent', trust_remote_code=True)
learning_buddy.run("Why is the sky blue?")